In [ ]:
import configparser
import boto3
import json

In [ ]:
# Read local config file
config = configparser.ConfigParser()

def read_all_keys(path_to_config):
    config.read_file(open(path_to_config))
    for section in config.sections():
        for key in config[section]:
            print(f'{key}: {config[section][key]}')
            
def write_key(path_to_config, section, key, value):
    config[section][key] = value
    with open(path_to_config, 'w') as configfile:
        config.write(configfile)
        
read_all_keys('dwh.cfg')

# Create IAM role for Redshift

In [ ]:
iam = boto3.client('iam', aws_access_key_id=config.get('KEYS', 'access_key'), 
                   aws_secret_access_key=config.get('KEYS', 'secrect_access_key'),
                   region_name='us-west-2')

In [ ]:
def create_redshift_iam_role(role_name, description):
    
    # Create role
    dwh_role = iam.create_role(
        Path='/',
        RoleName=role_name,
        Description=description,
        AssumeRolePolicyDocument=json.dumps(
        {'Statement': [{'Action': 'sts:AssumeRole',
        'Effect': 'Allow',
        'Principal': {'Service': 'redshift.amazonaws.com'}}],
        'Version': '2012-10-17'})
    )
    
    # Attach policy
    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    )['ResponseMetadata']['HTTPStatusCode']
    
    # Get role arn
    role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    return role_arn

In [ ]:
name = 'my-redshift-udacity-role'
desc = 'Allows Redshift clusters to call AWS services on your behalf.'

arn = create_redshift_iam_role(name, desc)
arn

### Update config file

In [ ]:
write_key('dwh.cfg', 'IAM_ROLE', 'arn', arn)

# Create Redshift Cluster

In [ ]:
redshift = boto3.client('redshift', aws_access_key_id=config.get('KEYS', 'access_key'), 
                        aws_secret_access_key=config.get('KEYS', 'secrect_access_key'),
                        region_name='us-west-2')

In [ ]:
cluster_type = 'multi-node'
node_type = 'dc2.large'
num_nodes = 4
cluster_identifier = 'my-redshift-udacity-cluster-1'

db = config.get('CLUSTER', 'db_name')
db_user = config.get('CLUSTER', 'db_user')
db_password = config.get('CLUSTER', 'db_password')
redshift_iam_role = config.get('IAM_ROLE', 'arn')

redshift.create_cluster(
    ClusterType=cluster_type,
    NodeType=node_type,
    NumberOfNodes=int(num_nodes),

    #Identifiers & Credentials
    DBName=db,
    ClusterIdentifier=cluster_identifier,
    MasterUsername=db_user,
    MasterUserPassword=db_password,

    #Roles (for s3 access)
    IamRoles=[redshift_iam_role]  
)

In [ ]:
def describe_redshift_cluster(props):
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return x

In [ ]:
props = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)['Clusters'][0]
describe_redshift_cluster(props)

In [ ]:
host = props['Endpoint']['Address']
host

### Update config file

In [ ]:
write_key('dwh.cfg', 'CLUSTER', 'host', host)

# Connect to Cluster and Run queries

In [ ]:
%load_ext sql

In [ ]:
db = config.get('CLUSTER', 'db_name')
db_user = config.get('CLUSTER', 'db_user')
db_password = config.get('CLUSTER', 'db_password')
endpoint = config.get('CLUSTER', 'host')
port = config.get('CLUSTER', 'db_port')

redshift_iam_role = config.get('IAM_ROLE', 'arn')

conn_string="postgresql://{}:{}@{}:{}/{}".format(db_user, db_password, endpoint, port, db)
print(conn_string)
%sql $conn_string

In [ ]:
%%sql

select count(*)
from staging_events;

In [ ]:
%%sql

select count(*)
from staging_songs;

In [ ]:
%%sql

select
    a.artist_name,
    s.title,
    t.month,
    count(*) as listened_count
from songplays sp
join artists a
    on a.artist_id = sp.artist_id
join songs s
    on s.song_id = sp.song_id
join times t
    on t.start_time = sp.start_time
group by
    a.artist_name,
    s.title,
    t.month
order by listened_count desc
limit 10;

# Delete Redshift Cluster

In [ ]:
redshift.delete_cluster(ClusterIdentifier=cluster_identifier, SkipFinalClusterSnapshot=True)